In [40]:
%%file Arthur_service_dao.py
# -*- coding: utf-8 -*-
# author: leepand
# time: 2018-12-17
# desc: tb_algo_dao


'''
#算法管理表
DROP TABLE IF EXISTS `tb_algo`;
CREATE TABLE `tb_algo` (
`aid` int(10) NOT NULL AUTO_INCREMENT,
`uid` int(10) NOT NULL,
`algoname` varchar(50) NOT NULL,
`algodesc` varchar(50) NOT NULL,
`version` varchar(50) NOT NULL  DEFAULT 'v1.0.0',
`opertype` enum('publish','register')  DEFAULT 'publish',
`token` varchar(50) NOT NULL,
`pyfile` varchar(50) NOT NULL,
`funcslist` text NOT NULL,
`tags` text NOT NULL,
`field` int(10) NOT NULL,
`host` varchar(50) NOT NULL  DEFAULT '0.0.0.0',
`port` int(10) NOT NULL,
`atype` varchar(10) NOT NULL DEFAULT 'REST',
`algo_tm` datetime NOT NULL,
`status` enum('normal','stop','notpublish') DEFAULT 'notpublish',
`is_email` varchar(10) NOT NULL DEFAULT 'no',
`remark` text NOT NULL,
`insert_tm` datetime NOT NULL,
PRIMARY KEY (`aid`),
KEY `uid` (`uid`),
CONSTRAINT `tb_algo_ibfk_1` FOREIGN KEY (`uid`) REFERENCES `tb_user` (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''



if __name__ == "__main__":
    import sys
    sys.path.append("..")
    sys.path.append("../base")

from hjs_cfg import *
from bs_util import *
from bs_database_pid import *
from bs_time import *

class ArthurAlgoDao:

    @staticmethod
    def insert_node(uId, algoname,algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark):

        dataBase = DataBase()
        sql = "insert into tb_algo(uid, algoname, algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark, insert_tm) " \
              "values(%s, %s, %s, %s, %s, %s,%s, %s, %s, %s, %s, %s,%s, %s,%s,%s)"
        param = (uId, algoname,algodesc,opertype,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm,is_email, remark, get_cur_time())

        bRet, sRet = dataBase.insert_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node(aId, atype, algo_tm,status, remark):

        dataBase = DataBase()
        sql = "update tb_algo set atype=%s, algo_tm=%s, " \
              "status = %s, remark=%s where aid= %s"
        param = (aotype, algo_tm, status, remark, aId)

        bRet, sRet = dataBase.update_data(sql, param)
        return bRet, sRet


    @staticmethod
    def update_node_status(aId, status):
        dataBase = DataBase()
        updatetime=get_cur_day(0, format="%Y-%m-%d %H:%M:%S")
        #sql = "update tb_order set status = %s where oid = %s"
        #param = (status, oId)
        sql = "update tb_algo set status = %s,insert_tm = %s where aid = %s"
        param = (status,updatetime,aId)
        bRet, sRet = dataBase.update_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_list(offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_algo where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_count(status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where 1=1 "
        param = []

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']
    #用户的服务list查询
    @staticmethod
    def query_node_list_by_uid(uId,offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_algo where uid=%s "
        param = [uId]
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet
    #用户的服务list查询
    @staticmethod
    def query_node_count_by_uid(uId,status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where uid=%s "
        param = [uId]

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']
    @staticmethod
    def query_node_by_oid(oId):
        dataBase = DataBase()
        sql = "select * from tb_order where oid = %s"
        param = (oId, )

        bRet, sRet = dataBase.query_data(sql, param)
        if (not bRet) or (len(sRet) != 1):
            return False, sRet
        
        return True, sRet[0]


    @staticmethod
    def query_node_by_status(status):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_order where 1=1 "
        param = []

        if status == 'all':
            pass
        else:
            sql += 'and status = %s'
            param.append(status)
        
        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']

        
    @staticmethod
    def query_node_by_days(days):
        dataBase = DataBase()
        sql = "select oid, cid, name, date_format(order_tm, '%%Y-%%m-%%d') as end_tm from tb_order " \
              "where order_tm >= curdate() and order_tm < curdate() + %s"
        param = (days, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet

    
    @staticmethod
    def query_node_by_date(status, tg_date):
        dataBase = DataBase()
        #sql = "select * from tb_algo where status = %s and algo_tm <= %s and algo_tm >= %s"
        sql = "select * from tb_algo where status = %s and insert_tm <= %s and insert_tm >= %s"
        sql_stop = "select * from tb_algo where status = %s and insert_tm <= %s and insert_tm >= %s"
        param = (status,tg_date,get_cur_day(0, format="%Y-%m-%d"))
        if status=='normal':
            bRet, sRet = dataBase.query_data(sql, param)
        else:
            bRet, sRet = dataBase.query_data(sql_stop, param)
        if not bRet:
            return False, sRet

        return True, sRet

    @staticmethod
    def query_node_by_date2(status, tg_date):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_order where status = %s and order_tm <= %s and order_tm >= %s"
        sql_stop = "select count(*) as cnt from tb_order where status = %s and insert_tm <= %s and insert_tm >= %s"
        param = (status, tg_date,get_cur_day(0, format="%Y-%m-%d"))

        if status=='normal':
            bRet, sRet = dataBase.query_data(sql, param)
        else:
            bRet, sRet = dataBase.query_data(sql_stop, param)
        if not bRet:
            return False, sRet
        if len(sRet) != 1:
            return True, 0
        
        return True, sRet[0]['cnt']

if __name__ == "__main__":
    #print HjsOrderDao.query_node_by_status('stop')
    import datetime
    today = datetime.date.today()
    date_t=today.isoformat()
    uId=41
    algoname='leepand'
    pyfile='./predict.py'
    funcslist='predict,pre2'
    algodesc='text mining'
    tags='nlp'
    token='sdksadfksdf'
    field=41
    atype='recom'
    host='0.0.0.0'
    is_email='no'
    port=9000
    algo_tm= "2018-03-12 12:23:11"
    #order_tm=date_t
    
    remark='just test'
    ArthurAlgoDao.update_node_status(1,'normal')
    #print HjsOrderDao.insert_node(cId, name, otype, order_tm, remark)
    #print HjsOrderDao.query_node_by_date2('normal', '2019-01-02')
    #ArthurAlgoDao.insert_node(uId, algoname,algodesc,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark)

#uId, algoname,algodesc,token,pyfile,funcslist,tags,field,host,port,atype, algo_tm, is_email,remark

#uId, algoname,algodesc,token,pyfile,funcslist,tags,host,port,atype, algo_tm, is_email,remark


Overwriting Arthur_service_dao.py


In [34]:
%%file Arthur_ps_service_dao.py
# -*- coding: utf-8 -*-

# author: leepand
# time: 2018-12-16
# desc: tb_service_dao


'''
# 暂取消服务表
CREATE TABLE `tb_ps_algo` (
`pid` int(11) NOT NULL AUTO_INCREMENT,
`aid` int(10) NOT NULL,
`uid` int(10) NOT NULL,
`algoname` varchar(50) NOT NULL,
`pause_tm` date NOT NULL,
`remark` text NOT NULL,
`insert_tm` datetime NOT NULL,
PRIMARY KEY (`pid`),
KEY `aid` (`aid`),
KEY `uid` (`uid`),
CONSTRAINT `tb_ps_algo_ibfk_1` FOREIGN KEY (`aid`) REFERENCES `tb_algo` (`aid`),
CONSTRAINT `tb_ps_algo_ibfk_2` FOREIGN KEY (`uid`) REFERENCES `tb_user` (`uid`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8;
'''

if __name__ == "__main__":
    import sys
    sys.path.append("..")
    sys.path.append("../base")

from hjs_cfg import *
from bs_util import *
from bs_database_pid import *


class ArthurServicePauseDao:

    @staticmethod
    def insert_node(aid, uid, algoname, pause_tm, remark):

        dataBase = DataBase()
        sql = "insert into tb_ps_algo(aid, uid, algoname, pause_tm, remark, insert_tm) " \
              "values(%s, %s, %s, %s, %s, %s)"
        param = (aid, uid, algoname, pause_tm, remark, get_cur_time())

        bRet, sRet = dataBase.insert_data(sql, param)
        return bRet, sRet

    @staticmethod
    def query_node_list(offset, limit, status, search):
        dataBase = DataBase()
        sql = "select * from tb_algo where 1=1 "
        param = []
        if status and status != 'all':
            sql += "and status = %s "
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        sql += "order by aid desc limit %s, %s"
        param.append(offset)
        param.append(limit)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet

        return True, sRet


    @staticmethod
    def query_node_count(status=None, search=None):
        dataBase = DataBase()
        sql = "select count(*) as cnt from tb_algo where 1=1 "
        param = []

        if status and status != 'all':
            sql += "and status = %s"
            param.append(status)
        if search:
            search = "%%%s%%" % (search)
            sql += "and (uid like %s or algoname like %s)"
            param.append(search)
            param.append(search)

        param = tuple(param)
        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
        if len(sRet) !=1:
            return True, 0

        return True, sRet[0]['cnt']


    @staticmethod
    def delete_node_by_pid(pId):
        dataBase = DataBase()
        sql = "delete from tb_ps_algo where pid = %s"
        param = (pId, )

        bRet, sRet = dataBase.delete_data(sql, param)
        if not bRet:
            return False, sRet
        
        return True, sRet


    @staticmethod
    def query_node_by_date(tg_date):
        dataBase = DataBase()
        sql = "select * from tb_ps_algo where pause_tm = %s"
        param = (tg_date, )

        bRet, sRet = dataBase.query_data(sql, param)
        if not bRet:
            return False, sRet
    
        return True, sRet



if __name__ == "__main__":
    #print HjsOrderDao.query_node_by_status('stop')

    #print HjsOrderDao.query_node_by_date(3)
    print ArthurServicePauseDao.delete_node_by_pid(3)







Writing Arthur_ps_service_dao.py


In [5]:
# 管道执行命令， 命令小输出还可以， 如果输出比较大推荐走文件输出流
def exec_cmd(cmd):
    
    Log.debug("cmd: %s" %(cmd))
    content = ""
    try:
        p = Popen(cmd, bufsize = 4096, stdout = PIPE, shell = True)
        
        while True:
            cont = p.stdout.read()
            if cont == "": break
            content += cont
            Log.debug("contLen: %d" %(len(cont)))
            time.sleep(1)
        retState = p.wait()
        
        return retState, content
    except Exception, e:
        Log.err("(%s)" %(traceback.format_exc()))
        return 255, "cmd(%s) err: %s" %(str(cmd), str(e))

In [21]:
from subprocess import *
cmd='pwd'
exec_cmd(cmd)

[18/12/2018 08:12:58][DEBUG]<ipython-input-18-45c44df60504>::exec_cmd(4): cmd: pwd
[18/12/2018 08:12:58][DEBUG]<ipython-input-18-45c44df60504>::exec_cmd(13): contLen: 68


(0, '/Users/leepand/Downloads/recom/python_web/Arthur_deploy_web/src/dao\n')

In [1]:
class ShellCommandException(Exception):
    pass

import subprocess
def exec_cmd(cmd, throw_on_error=True, env=None, stream_output=False, cwd=None, cmd_stdin=None,
             **kwargs):
    """
    Runs a command as a child process.
    A convenience wrapper for running a command from a Python script.
    Keyword arguments:
    cmd -- the command to run, as a list of strings
    throw_on_error -- if true, raises an Exception if the exit code of the program is nonzero
    env -- additional environment variables to be defined when running the child process
    cwd -- working directory for child process
    stream_output -- if true, does not capture standard output and error; if false, captures these
      streams and returns them
    cmd_stdin -- if specified, passes the specified string as stdin to the child process.
    Note on the return value: If stream_output is true, then only the exit code is returned. If
    stream_output is false, then a tuple of the exit code, standard output and standard error is
    returned.
    """
    cmd_env = os.environ.copy()
    if env:
        cmd_env.update(env)

    if stream_output:
        child = subprocess.Popen(cmd, env=cmd_env, cwd=cwd, universal_newlines=True,
                                 stdin=subprocess.PIPE, **kwargs)
        child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exitcode: %s" % (exit_code))
        return exit_code
    else:
        child = subprocess.Popen(
            cmd, env=cmd_env, stdout=subprocess.PIPE, stdin=subprocess.PIPE, stderr=subprocess.PIPE,
            cwd=cwd, universal_newlines=True, **kwargs)
        (stdout, stderr) = child.communicate(cmd_stdin)
        exit_code = child.wait()
        if throw_on_error and exit_code is not 0:
            raise ShellCommandException("Non-zero exit code: %s\n\nSTDOUT:\n%s\n\nSTDERR:%s" %
                                        (exit_code, stdout, stderr))
        return exit_code, stdout, stderr
__version__ = '0.0.7'
APIFLY_FUNCTIONS="APIFLY_FUNCTIONS"
APIFLY_TOKEN="APIFLY_TOKEN"
def _run_server(apifly_function, apifly_token, host, port, workers=2, is_reg=False,static_prefix=None,
                gunicorn_opts=None):
    """
    Run the Arthur api server, wrapping it in gunicorn
    :param static_prefix: If set, the index.html asset will be served from the path static_prefix.
                          If left None, the index.html asset will be served from the root path.
    :return: None
    """
    env_map = {}
    if apifly_function:
        env_map[APIFLY_FUNCTIONS] = apifly_function
    if apifly_token:
        env_map[APIFLY_TOKEN] = apifly_token
    if static_prefix:
        env_map[STATIC_PREFIX_ENV_VAR] = static_prefix

    bind_address = "%s:%s" % (host, port)
    bind_address2 = "http//%s:%s" % (host, port)
    if is_reg:
        redis_db=api.data_api(redis_db=3).redis_api(use_type='mem',prefix='REST:SERVICE_LIST:')
        reg_info={}
        funcs_info=load_functions(apifly_function.split(","))
        addr=bind_address2
        token=apifly_token
        reg_info['addr']=addr
        reg_info['token']=token
        reg_info['funcs_list']=funcs_info
        redis_db['REST_SERVICE_REG']=json.dumps(reg_info)
    opts = shlex.split(gunicorn_opts) if gunicorn_opts else []
    return exec_cmd(["gunicorn"] + opts + ["-b", bind_address, "-w", "%s" % workers,
                                    "--worker-class","gevent",
                                    "Arthur.api_server.main:app"],env=env_map, stream_output=True)

In [22]:
%%file  fib.py

def fib(n):
	if n == 0 or n == 1:
		return 1
	else:
		return fib(n-1) + fib(n-2)

Writing fib.py


In [1]:
import os
#x=_run_server('fib.fib', '', '0.0.0.0', 9069)
import fib

In [23]:
from subprocess import *
cmd='nohup Arthur apiserver -f fib.fib >run.log 2>&1 &'
x=exec_cmd(cmd)

[18/12/2018 10:25:08][DEBUG]<ipython-input-5-45c44df60504>::exec_cmd(4): cmd: nohup Arthur apiserver -f fib.fib >run.log 2>&1 &


In [30]:
wait_until_process_used_ports(
    'Arthur', [5000], max_wait_sec=10, interval_sec=0.5
)

TypeError: 'int' object is not iterable

In [25]:
exec_cmd('lsof -i:5000')

[18/12/2018 10:25:52][DEBUG]<ipython-input-5-45c44df60504>::exec_cmd(4): cmd: lsof -i:5000
[18/12/2018 10:25:53][DEBUG]<ipython-input-5-45c44df60504>::exec_cmd(13): contLen: 1910


(0,
 'COMMAND     PID    USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME\npython2.7 24159 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24162 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24163 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24164 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24165 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24166 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24167 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24168 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (LISTEN)\npython2.7 24169 leepand   62u  IPv4 0x2de2d023ed63682b      0t0  TCP localhost:commplex-main (L

In [14]:
%%file saferun
#!/bin/bash

CMDLINE=$@

echo `date +"%Y-%m-%d %H:%M:%S"`" begin to start:$CMDLINE"

$CMDLINE &
PID="$!"

echo `date +"%Y-%m-%d %H:%M:%S"`" has started pid:$PID, $CMDLINE"
while true
do
    wait $PID
    echo `date +"%Y-%m-%d %H:%M:%S"`" terminated, to start again"
    $CMDLINE &
    PID="$!"
    echo `date +"%Y-%m-%d %H:%M:%S"`" restarted pid:$PID, $CMDLINE"
done

Writing saferun


In [18]:
__all__ = [
    'wait_until_file_exist', 'wait_until_reg_str_exist',
    "wait_until_process_not_exist", "wait_until_port_used",
    "wait_until_process_used_ports", "wait_until_port_not_used",
    "wait_until_process_exist", "wait_until_process_killed"
]

import os
import re
import time

import cup
from cup.shell import oper


def wait_until_file_exist(
    dst_path, file_name, max_wait_sec=10, interval_sec=2, recursive=False
):
    """
    wait util the file exists or the function timeout
    :param dst_path:
        searching path
    :param file_name:
        filename, support *
    :param max_wait_sec:
        max wating time until timeout
    :param interval_sec:
        check interval
    :param recursive:
        recursively search or not
    :return:
        True if found.
    """
    curr_wait_sec = 0

    while curr_wait_sec < max_wait_sec:
        if oper.contains_file(dst_path, file_name, recursive):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_reg_str_exist(
    dst_file_path, reg_str, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until any line in the file matches the \
    reg_str(regular expression string)
    :param dst_file_path:
        searching path
    :param reg_str:
        regular expression string
    :param max_wait_sec:
        maximum waiting time until timeout
    :param interval_sec:
        state check interval
    :return:
        True if found
    """
    curr_wait_sec = 0
    file_reader = FileReader(dst_file_path)
    while curr_wait_sec < max_wait_sec:
        if __check_reg_str_contain(file_reader, reg_str):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_process_not_exist(
    process_path, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the process does not exist anymore or the function timeouts
    :param process_path:
        process cwd
    :param max_wait_sec:
        maximum waiting time until timeout. 10 seconds by default
    :param interval_sec:
        state check interval, 0.5 second by default
    :return:
        return True if the process disapper before timeout
    """
    process_path = os.path.abspath(process_path)
    pro_path = os.path.dirname(process_path)
    pro_name = os.path.basename(process_path)
    curr_wait_sec = 0
    while curr_wait_sec < max_wait_sec:
        if not oper.is_proc_exist(pro_path, pro_name):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_process_exist(
    process_path, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the process exists
    :param process_path:
        the specific process working path
    :param max_wait_sec:
        maximum waiting time until timeout
    :param interval_sec:
        state check interval
    :return:
        return True if the process is found before timeout
    """
    process_path = os.path.abspath(process_path)
    pro_path = os.path.dirname(process_path)
    pro_name = os.path.basename(process_path)
    curr_wait_sec = 0
    while curr_wait_sec < max_wait_sec:
        if oper.is_proc_exist(pro_path, pro_name):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_port_used(
    port, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the port is used.  *Notice this function will invoke\
    a bash shell to execute command [netstat]!*
    :return:
        return True if the port is used
    """
    curr_wait_sec = 0

    while curr_wait_sec < max_wait_sec:
        if oper.is_port_used(port):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_port_not_used(
    port, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the port is free
    :return:
        return True if the port is free before timeout
    """
    curr_wait_sec = 0

    while curr_wait_sec < max_wait_sec:
        if not oper.is_port_used(port):
            return True
        curr_wait_sec += interval_sec
        time.sleep(interval_sec)
    return False


def wait_until_process_used_ports(
    process_path, ports, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the process has taken the ports before timeouts
    :return:
        True if all ports are used by the specific process.
        False, otherwise
    """
    curr_wait_sec = 0

    while curr_wait_sec < max_wait_sec:
        used_port_num = 0
        for curr_port in ports:
            if oper.is_process_used_port(process_path, curr_port):
                used_port_num += 1
            else:
                break
        if used_port_num != len(ports):
            curr_wait_sec += interval_sec
            time.sleep(interval_sec)
        else:
            return True
    return False


def wait_until_process_killed(
    process_path, ports, max_wait_sec=10, interval_sec=0.5
):
    """
    wait until the [process] does not exists and all [ports] are free
    :param process_path:
        process cwd
    :param ports:
        port list
    :param interval_sec:
        state check interval
    :return:
        True if all conditions meet.
    """
    curr_wait_sec = 0
    while curr_wait_sec < max_wait_sec:
        # check process
        if False == wait_until_process_not_exist(
            process_path, max_wait_sec, interval_sec
        ):
            curr_wait_sec += interval_sec
            time.sleep(interval_sec)
            continue
        # check ports
        not_used_port_num = 0
        for curr_port in ports:
            if not oper.is_process_used_port(process_path, curr_port):
                not_used_port_num += 1
            else:
                break
        if not_used_port_num != len(ports):
            curr_wait_sec += interval_sec
            time.sleep(interval_sec)
        else:
            return True
    return False


def _wait_until_return(func,
        boolean, max_wait_sec, interval_sec=0.5, *args, **kwargs
    ):
    """
    wait until function return [boolean]
    """
    curr_wait_sec = 0

    while curr_wait_sec < max_wait_sec:
        if func(*args, **kwargs) == boolean:
            return True
        else:
            time.sleep(interval_sec)
            curr_wait_sec += interval_sec
    return False


def wait_return_true(func, max_wait_sec, interval_sec=0.5, *args, **kwargs):
    """
    wait until func return true or max_wait_sec passes.
    """
    return _wait_until_return(
        func, True, max_wait_sec, interval_sec, *args, **kwargs
    )


def wait_return_false(func, max_wait_sec, interval_sec=0.5, *args, **kwargs):
    """
    wait until func return False or max_wait_sec passes.
    """
    return _wait_until_return(
        func, False, max_wait_sec, interval_sec, *args, **kwargs
    )


def __check_reg_str_contain(file_reader, reg_str):
    """
    check if any line matches the reg_str
    :param file_reader:
        FileReade Object
    :return:
        return True if found
    """
    file_content = file_reader.read()
    lines = file_content.splitlines()
    for line in lines:
        if re.search(reg_str, line):
            return True
    return False


class FileReader(object):
    """
    this class is used to read file incremental
    """
    def __init__(self, file_path):
        self.file_path = file_path
        self.pos = 0

    def read(self, max_read_size=None):
        """
        read from last position
        :param max_read_size:
            maximum reading length
        :return:
            content read
        """
        # whether the file exist
        if not os.path.isfile(self.file_path):
            return ""
        ret = ""
        with open(self.file_path) as fp:
            fp.seek(0, os.SEEK_END)
            size = fp.tell()
            if size >= self.pos:
                fp.seek(self.pos, os.SEEK_SET)
                if (max_read_size is None) or (max_read_size > (size - self.pos)):
                    max_read_size = size - self.pos
                ret = fp.read(max_read_size)
                self.pos = self.pos + len(ret)
            else:  # may be a new file with the same name
                fp.seek(0, os.SEEK_SET)
                if (max_read_size is None) or (max_read_size > size):
                    max_read_size = size
                ret = fp.read(max_read_size)
                self.pos = len(ret)
        return ret

In [26]:
wait_until_reg_str_exist(
    'run.log', 'Starting apiserver', max_wait_sec=10, interval_sec=0.5)

False

In [27]:
wait_until_reg_str_exist(
    'run.log', 'Connection in use', max_wait_sec=10, interval_sec=0.5)

True